# Данные

Прочитаем файл и посмотрим на тексты

In [1]:
!pwd

/home/ffrankusha/study/university/nlp-course/tasks/task 1


In [2]:
import gzip

filepath = '../../data/news.txt.gz'

with gzip.open(filepath, 'rt', encoding='utf-8') as f:
    content = f.readline()
    
print(content)

style	Rolex наградит победителей регаты	Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы от швейцарского бренда Rolex. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру» в среду, 8 мая. Rolex Yacht-Master 40 Фото: пресс-служба Mercury Соревнования будут проходить с 10 по 18 июня. Первый этап: ночной переход из Сан-Ремо в Сен-Тропе 10-11 июня (дистанция 50 морских миль — около 90 километров). Второй этап: серия прибрежных гонок в бухте Сен-Тропе с 11 по 14 июня. Финальный этап пройдет с 15 по 18 июня: оффшорная гонка по маршруту Сен-Тропе — Генуя (243 морских мили — 450 километров). Маршрут проходит через скалистый остров Джиралья к северу от Корсики и завершается в Генуе.Регата, с 1997 года проходящая при поддержке Rolex, считается одной из самых значительных яхтенных гонок в Средиземноморье. В этом году в ней ожидается

Данные имеют формат `<tag>  <title>  <content>`

In [3]:
# import sys


# print(len(content))

# memory_size = sys.getsizeof(content)
# # в МБ
# memory_size / (1024 * 1024)

In [4]:
import pandas as pd

data = pd.read_csv(filepath,
                   sep='\t',
                   header=None)
data.columns = ['tag', 'title', 'content']

data.head(3)

,tag,title,content
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г..."


In [5]:
data.describe()

,tag,title,content
count,10000,10000,10000
unique,10,9995,10000
top,media,ЦБ отозвал лицензии у двух банков,Следователи назначили судебно-психиатрическую ...
freq,1476,3,1


# Описание facts, rules

Необходимо извлекать данные вида: 
* человек -> родился (родилось) в ... году
* человек -> родился (родилось) в городе/стране ...

In [6]:
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, is_capitalized, dictionary, normalized, gte, lte, caseless
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.relations import gnc_relation

gnc = gnc_relation()

### Facts

In [7]:
# Целевой факт
Entry = fact(
    'Entry',
    ['name', 'birth_date', 'birth_place']
)

# Человек (имя)
Person = fact(
    "Person",
    ["first_name", "last_name"]
)

# Дата рождения
BirthDate = fact(
    'Date',
    ['year', 'month', 'day']
)

# Место рождения
BirthPlace = fact(
    'Place',
    ['name']
)

### Правила для извлечения имени

In [8]:
# NAME = rule(
#     is_capitalized().match(gnc).interpretation(Person.first_name.inflected()).optional(),
#     is_capitalized().match(gnc).interpretation(Person.last_name.inflected()),
# ).interpretation(
#     Entry.name
# )

# более строгое
NAME_FIRST_LAST = rule(
    gram("Name").match(gnc).interpretation(Person.first_name.inflected()),
    gram("Surn").optional().match(gnc).interpretation(Person.last_name.inflected())
)

NAME_LAST_FIRST = rule(
    gram("Surn").optional().match(gnc).interpretation(Person.last_name.inflected()),
    gram("Name").match(gnc).interpretation(Person.first_name.inflected())
)

# видимо придется добавить
NAME_COMMON = rule(
    is_capitalized().match(gnc).interpretation(Person.last_name.inflected()).optional(),
    gram("Name").match(gnc).interpretation(Person.first_name.inflected()),
)
NAME_COMMON_REVERSE = rule(
    gram("Name").match(gnc).interpretation(Person.first_name.inflected()),
    is_capitalized().match(gnc).interpretation(Person.last_name.inflected()).optional()
)


NAME = or_(
    NAME_FIRST_LAST,
    NAME_LAST_FIRST,
    NAME_COMMON,
    NAME_COMMON_REVERSE
).interpretation(
    Person
)

In [9]:
# проверка
parser_name = Parser(NAME)

text = "Иван играл в футбол. Игорь Владимиров пришел домой. Иванов Петя родился в 2001 году."

for match in parser_name.findall(text):
    print([_.value for _ in match.tokens])
    print(match.fact)

['Иван']
Person(first_name='иван', last_name=None)
['Игорь', 'Владимиров']
Person(first_name='игорь', last_name='владимиров')
['Иванов', 'Петя']
Person(first_name='петя', last_name='иванов')


### Правила для извлечения даты рождения

In [10]:
# месяц может быть задан в словесном и числовом виде
months_names = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(
    months_names
).interpretation(
    BirthDate.month.normalized().custom(months_names.get)
)

MONTH_NUMBER = and_(
    gte(1),
    lte(12)
).interpretation(
    BirthDate.month.custom(int)
)

# объединение правил
MONTH = or_(
    MONTH_NAME,
    MONTH_NUMBER
)

# правило для дня
DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    BirthDate.day.custom(int)
)

# правило для года
YEAR = rule(
    and_(
        gte(1000),
        lte(2025)
    ).interpretation(
    BirthDate.year.custom(int)
    ),
    normalized("год").optional(),
    rule(caseless("г"), ".").optional()
)

DATE_YEAR = rule(
    DAY.optional(),
    dictionary({'.', '-', '/'}).optional(),
    MONTH.optional(),
    dictionary({'.', '-', '/'}).optional(),
    YEAR
)
DATE_DAY_MONTH = rule(
    DAY,
    dictionary({'.', '-', '/'}).optional(),
    MONTH,
    YEAR.optional()
)
DATE = or_(
        DATE_DAY_MONTH,
        DATE_YEAR  
)

BIRTH_DATE = rule(
    dictionary({"в"}).optional(),
    gram('ADJF').repeatable().optional(), # на случай прилагательных
    DATE
).interpretation(
        BirthDate
)

In [11]:
# проверка
parser_birth_date = Parser(BIRTH_DATE)

text = "Иван родился в 10-05-1980. У Игоря день рождения 20/04/1990. Петя родился в 2001 году. У Яны день рождения 21 октября. Родился в 2000 г."

for match in parser_birth_date.findall(text):
    print([_.value for _ in match.tokens])
    print(match.fact)

['в', '10', '-', '05', '-', '1980']
Date(year=1980, month=5, day=10)
['20', '/', '04', '/', '1990']
Date(year=1990, month=4, day=20)
['в', '2001', 'году']
Date(year=2001, month=None, day=None)
['21', 'октября']
Date(year=None, month=10, day=21)
['в', '2000', 'г', '.']
Date(year=2000, month=None, day=None)


### Правила для извлечения места рождения

In [12]:
BIRTH_PLACE = rule(
    dictionary({'в', 'из', 'на'}), 
    gram('ADJF').optional().repeatable(),  # прилагательное для города
    dictionary({'город', 'страна', 'место', 'республика', 'край', 'село', 'деревня', 'округ', 'область', 'район'}).optional(),
    and_(is_capitalized(),
         gram('NOUN')).interpretation(BirthPlace.name.inflected())           # город
).interpretation(BirthPlace)

In [13]:
# проверка
parser_birth_place = Parser(BIRTH_PLACE)

text = "Иван родился в прекрасном городе Кострома. Игорь родом из города Самара. Петр родился в прекрасной стране Россия"

for match in parser_birth_place.findall(text):
    print([_.value for _ in match.tokens])
    print(match.fact)

['в', 'прекрасном', 'городе', 'Кострома']
Place(name='кострома')
['из', 'города', 'Самара']
Place(name='самара')
['в', 'прекрасной', 'стране', 'Россия']
Place(name='россия')


### Финальное правило

In [14]:
MAIN_VERB = dictionary({'родился', 'родом', 'рождение'})

ENTRY = or_(
            rule(NAME.interpretation(Entry.name),
                 MAIN_VERB, 
                 BIRTH_DATE.interpretation(Entry.birth_date),
                 MAIN_VERB.optional(), 
                 BIRTH_PLACE.interpretation(Entry.birth_place)),
            rule(NAME.interpretation(Entry.name),
                 MAIN_VERB, 
                 BIRTH_PLACE.interpretation(Entry.birth_place), 
                 MAIN_VERB.optional(), 
                 BIRTH_DATE.interpretation(Entry.birth_date)),
            rule(MAIN_VERB, 
                 BIRTH_DATE.interpretation(Entry.birth_date), 
                 NAME.interpretation(Entry.name), 
                 MAIN_VERB.optional(), 
                 BIRTH_PLACE.interpretation(Entry.birth_place)),
            rule(MAIN_VERB, 
                 BIRTH_PLACE.interpretation(Entry.birth_place), 
                 NAME.interpretation(Entry.name), 
                 MAIN_VERB.optional(), 
                 BIRTH_DATE.interpretation(Entry.birth_date)),
            rule(MAIN_VERB, 
                 BIRTH_PLACE.interpretation(Entry.birth_place), 
                 BIRTH_DATE.interpretation(Entry.birth_date), 
                 NAME.interpretation(Entry.name)),
            rule(MAIN_VERB, 
                 BIRTH_DATE.interpretation(Entry.birth_date), 
                 BIRTH_PLACE.interpretation(Entry.birth_place), 
                 NAME.interpretation(Entry.name))
    ).interpretation(
        Entry
    )

parser = Parser(ENTRY)

In [ ]:
# проверка
text = "Иван Валентинов родился 21 сентября 2001 года в Костроме. \
        Петя Петрович родился в России 10-10-2006. \
        Максим родился 19/10/1700 в городе Казань \
        Родившись в 2000 году в Подмосковье Петя Шумный ..."

for match in parser.findall(text):
    print(match.fact)

Entry(name=Person(first_name='иван', last_name='валентинов'), birth_date=Date(year=2001, month=9, day=21), birth_place=Place(name='кострома'))
Entry(name=Person(first_name='петя', last_name='петрович'), birth_date=Date(year=2006, month=10, day=10), birth_place=Place(name='россия'))
Entry(name=Person(first_name='максим', last_name=None), birth_date=Date(year=1700, month=10, day=19), birth_place=Place(name='казань'))
Entry(name=Person(first_name='петя', last_name='шумный'), birth_date=Date(year=2000, month=None, day=None), birth_place=Place(name='подмосковье'))


# Применение

In [16]:
import gzip

from dataclasses import dataclass
from typing import Iterator

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

filepath = '../../data/news.txt.gz'

texts = list(read_texts(filepath))

In [17]:
from tqdm import tqdm 

for text in tqdm(texts, disable=True):
    try:
        for match in parser.findall(text.text):
         print(match.fact)
    except:
       # empty
       pass

Entry(name=Person(first_name='бетси', last_name='палмера'), birth_date=Date(year=1926, month=None, day=None), birth_place=Place(name='сша'))
Entry(name=Person(first_name='трэмиел', last_name=None), birth_date=Date(year=1928, month=None, day=None), birth_place=Place(name='польша'))
Entry(name=Person(first_name='дмитрий', last_name='чернявский'), birth_date=Date(year=1992, month=3, day=5), birth_place=Place(name='артёмовск'))
Entry(name=Person(first_name='яковлевюрий', last_name='яковлев'), birth_date=Date(year=1928, month=None, day=None), birth_place=Place(name='москва'))
Entry(name=Person(first_name='патрик', last_name=None), birth_date=Date(year=1990, month=None, day=None), birth_place=Place(name='бронкс'))
Entry(name=Person(first_name='николай', last_name='караченцов'), birth_date=Date(year=1944, month=10, day=27), birth_place=Place(name='москва'))
Entry(name=Person(first_name='игорь', last_name='доценко'), birth_date=Date(year=1953, month=None, day=None), birth_place=Place(name='укр

24 совпадения

# Дополнение

В natasha есть [готовые функции](https://habr.com/ru/articles/349864/#:~:text=%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E%20Yargy%2D%D0%BF%D0%B0%D1%80%D1%81%D0%B5%D1%80%D0%B0.-,%D0%93%D0%BE%D1%82%D0%BE%D0%B2%D1%8B%D0%B5%20%D0%BF%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0,-%D0%A1%D0%B5%D0%B9%D1%87%D0%B0%D1%81%20%D0%B2%20Natasha) для извлечения популярных паттернов

In [18]:
from natasha import (
    NamesExtractor,
    AddressExtractor,
    DatesExtractor,
)

extractors = [
    NamesExtractor(),
    AddressExtractor(),
    DatesExtractor(),
]

ImportError: cannot import name 'AddressExtractor' from 'natasha' (/home/ffrankusha/study/university/nlp-course/venv/lib/python3.10/site-packages/natasha/__init__.py)

Возможно, их использование упростило бы задачу 😃